### The Johns Hopkins University: Building Machine Models to Predict Sepsis

## Business Understanding:
Goal: To develop a predictive model that accurately identifies patients at risk of developing sepsis.

Null Hypothesis (H₀): There is no significant relationship between the patient's medical data and the likelihood of developing sepsis.

Alternative Hypothesis (H₁): There is a significant relationship between the patient's medical data and the likelihood of developing sepsis.

#### Analytical questions:
1. What is the correlationship between age and sepsis?
2. What effect does the level of plasma glucose have on sepsis?
3. What is the effect of blood pressure on sepsis?
4. What is the relationship between body mass index and sepsis?
5. Does one insurance health affect sepsis?

### Features
1. ID: number to represent patient ID
2. PRG: Plasma glucose
3. PL: Blood Work Result-1 (mu U/ml)
4. PR: Blood Pressure (mm Hg)
5. SK: Blood Work Result-2 (mm)
6. TS: Blood Work Result-3 (mu U/ml)
7. M11: Body mass index (weight in kg/(height in m)^2
8. BD2: Blood Work Result-4 (mu U/ml)
9. Age: patients age (years)
10. Insurance: If a patient holds a valid insurance card
11. Sepsis: Positive: if a patient in ICU will develop a sepsis , and Negative: otherwise

# Import the required packages

In [ ]:
# Data Manipulation and Hypothesis testing
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr

# For saving the model, processor and cleaned data sets
import pickle
import joblib

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    RobustScaler, 
    LabelEncoder, 
    QuantileTransformer
)
from sklearn.model_selection import (
    train_test_split, 
    GridSearchCV, 
    RandomizedSearchCV,
    StratifiedKFold
)
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    recall_score,
    f1_score,
    precision_score,
    roc_auc_score,
    classification_report
)
from sklearn.ensemble import (
    AdaBoostClassifier, 
    BaggingClassifier, 
    ExtraTreesClassifier, 
    GradientBoostingClassifier, 
    RandomForestClassifier
)

# Utilities
import warnings
import joblib
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### Data Collections

In [ ]:
# Import the train data
train_data = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\API DOCKER\data\Paitients_Files_Train.csv')
print(train_data.head())

          ID  PRG   PL  PR  SK   TS   M11    BD2  Age  Insurance   Sepssis
0  ICU200010    6  148  72  35    0  33.6  0.627   50          0  Positive
1  ICU200011    1   85  66  29    0  26.6  0.351   31          0  Negative
2  ICU200012    8  183  64   0    0  23.3  0.672   32          1  Positive
3  ICU200013    1   89  66  23   94  28.1  0.167   21          1  Negative
4  ICU200014    0  137  40  35  168  43.1  2.288   33          1  Positive


In [ ]:
# Import the test data
test_data = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\API DOCKER\data\Paitients_Files_Test.csv')
print(test_data.head())

          ID  PRG   PL  PR  SK   TS   M11    BD2  Age  Insurance
0  ICU200609    1  109  38  18  120  23.1  0.407   26          1
1  ICU200610    1  108  88  19    0  27.1  0.400   24          1
2  ICU200611    6   96   0   0    0  23.7  0.190   28          1
3  ICU200612    1  124  74  36    0  27.8  0.100   30          1
4  ICU200613    7  150  78  29  126  35.2  0.692   54          0


### Exploratory Data Analysis (EDA) of Train Data Set

In [ ]:
# Check the info of train data
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         599 non-null    object 
 1   PRG        599 non-null    int64  
 2   PL         599 non-null    int64  
 3   PR         599 non-null    int64  
 4   SK         599 non-null    int64  
 5   TS         599 non-null    int64  
 6   M11        599 non-null    float64
 7   BD2        599 non-null    float64
 8   Age        599 non-null    int64  
 9   Insurance  599 non-null    int64  
 10  Sepssis    599 non-null    object 
dtypes: float64(2), int64(7), object(2)
memory usage: 51.6+ KB
None


In [ ]:
# Check the Shape of the Train Data Set
train_data.shape

(599, 11)


In [ ]:
# Check the five summary statistics of the train data
train_data.describe()

              PRG          PL          PR          SK          TS         M11  \
count  599.000000  599.000000  599.000000  599.000000  599.000000  599.000000   
mean     3.824708  120.153589   68.732888   20.562604   79.460768   31.920033   
std      3.362839   32.682364   19.335675   16.017622  116.576176    8.008227   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   64.000000    0.000000    0.000000   27.100000   
50%      3.000000  116.000000   70.000000   23.000000   36.000000   32.000000   
75%      6.000000  140.000000   80.000000   32.000000  123.500000   36.550000   
max     17.000000  198.000000  122.000000   99.000000  846.000000   67.100000   

              BD2         Age   Insurance  
count  599.000000  599.000000  599.000000  
mean     0.481187   33.290484    0.686144  
std      0.337552   11.828446    0.464447  
min      0.078000   21.000000    0.000000  
25%      0.248000   24.000000    0.000000  
50%   

In [ ]:
# Check the five-summary statistics for category columns of the train data
train_data.describe(include = 'object').T

        count unique        top freq
ID        599    599  ICU200010    1
Sepssis   599      2   Negative  391


In [ ]:
# Check for duplicates in the train data
train_data.duplicated().sum()

0


In [ ]:
# Check for missing values in the train data
train_data.isna().sum()

ID           0
PRG          0
PL           0
PR           0
SK           0
TS           0
M11          0
BD2          0
Age          0
Insurance    0
Sepssis      0
dtype: int64


In [ ]:
# Rename the columns to give descriptive meaning

cols = {
    'ID': 'Patient_ID',
    'PRG': 'Plasma_Glucose',
    'PL': 'Blood_Work_Result-1 (mu U/ml)',
    'PR': 'Blood_Pressure (mm Hg)',
    'SK': 'Blood_Work_Result-2 (mm)',
    'TS': 'Blood_Work_Result-3 (mu U/ml)',
    'M11': 'Body_Mass_Index (weight in kg/(height in m)^2',
    'BD2': 'Blood_Work_Result-4 (mu U/ml)',
    'Sepssis' : 'Sepsis'
}

train_data.rename(columns = cols, inplace = True)

# Check the data
print(train_data.head())

  Patient_ID  Plasma_Glucose  Blood_Work_Result-1 (mu U/ml)  \
0  ICU200010               6                            148   
1  ICU200011               1                             85   
2  ICU200012               8                            183   
3  ICU200013               1                             89   
4  ICU200014               0                            137   

   Blood_Pressure (mm Hg)  Blood_Work_Result-2 (mm)  \
0                      72                        35   
1                      66                        29   
2                      64                         0   
3                      66                        23   
4                      40                        35   

   Blood_Work_Result-3 (mu U/ml)  \
0                              0   
1                              0   
2                              0   
3                             94   
4                            168   

   Body_Mass_Index (weight in kg/(height in m)^2  \
0                    

### Univariate Analysis

In [ ]:
# A histgram plot showing the distribution of numerical columns

train_data.hist(
    figsize = (12, 10),
    color = 'skyblue',
    grid = False,
    bins = 15
)
plt.show();

In [ ]:
# Checking for outliers by ploting the box plot

plt.figure(figsize = (10, 8))
sns.boxplot(
    data = train_data,
    orient = 'h'
)
plt.xscale('log') # Some features have extreme large numbers
plt.title("Outlier Features")
plt.show();

In [ ]:
# Check for kde distribution

sns.kdeplot(
    data = train_data
)
plt.xscale('log') # Some features have extreme large values
plt.title('KDE Distribution of Numerical Columns')
plt.xlabel('Values')
plt.show();

In [ ]:
# Checking whether our target variable is balanced using count plot

ax = sns.countplot(
    data = train_data,
    x = 'Sepsis'
)
plt.xlabel('Class Target')
plt.ylabel('Frequency')
plt.title("A Count plot of the Class Target")

# Annotate the figures for each category
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width()/2, p.get_height()),
        ha = 'center',
        va = 'center'
    )
plt.show()

In [ ]:
# Check the columns for the train data set
train_data.columns

Index(['Patient_ID', 'Plasma_Glucose', 'Blood_Work_Result-1 (mu U/ml)',
       'Blood_Pressure (mm Hg)', 'Blood_Work_Result-2 (mm)',
       'Blood_Work_Result-3 (mu U/ml)',
       'Body_Mass_Index (weight in kg/(height in m)^2',
       'Blood_Work_Result-4 (mu U/ml)', 'Age', 'Insurance', 'Sepsis'],
      dtype='object')


### Bivariate Analysis

In [ ]:
# Scatterplot of Age vs Plasma Glucose
sns.scatterplot(
    data = train_data,
    x = 'Age',
    y = 'Plasma_Glucose'
)
plt.title('Relationship Between Age and Plasma Glucose')
plt.show();

In [ ]:
# A bar chart of Sepsis vs Blood_Work_Result-1 (mu U/ml)

ax = sns.barplot(
    data = train_data,
    x = 'Sepsis',
    y = 'Blood_Work_Result-1 (mu U/ml)',
    ci = None
)
plt.title("A Barchart of Sepsis vs Blood_Work_Result-1 (mu U/ml)")

# Annotate the values for each bar
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width() / 2, p.get_height()),
         ha = 'center',
        va = 'center'
    )
plt.show();

### Multi-variate Analysis

In [ ]:
# Get all the numerical columns

numerical_col = train_data.select_dtypes(exclude = 'object').columns

# Create a data frame for numerical columns only

data_num_col = train_data[numerical_col]

# Calculate the correlationship for the numrical data frame

corr = data_num_col.corr()

# Visualize the correlationship using heatmap

plt.figure(figsize = (12, 10))
sns.heatmap(corr, annot = True, cmap = 'coolwarm', fmt = '.2f')
plt.title("Correlationship Between Features", fontsize = 12)
plt.show();

In [ ]:
# Bivariate Analaysis using Pairplot, Features vs Target

plt.figure(figsize = (12, 10))
sns.pairplot(
    data = train_data,
    diag_kind = 'kde',
    hue = 'Sepsis'
)
plt.show();

### Analytical Questions

In [ ]:
# i. What is the correlationship between age and sepsis?
ax = sns.barplot(
    data = train_data,
    x = 'Sepsis',
    y = 'Age',
    ci = None
)
plt.title('A Barchart of Sepsis vs Age')
# Annotate
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width() /2, p.get_height()),
        ha = 'center',
        va = 'center'
    )
plt.show();

print(f"From the graph its clear that patients who are older are likely to have sepsis")

From the graph its clear that patients who are older are likely to have sepsis


In [ ]:
# ii. What effect does the level of plasma glucose have on sepsis?

sns.boxplot(
    data = train_data,
    x = 'Plasma_Glucose',
    y = 'Sepsis'
)
plt.show();

print(f"From the boxplot, pateints with higher plasma glucose are more likely to have sepsis")

From the boxplot, pateints with higher plasma glucose are more likely to have sepsis


In [ ]:
# iii. What is the effect of blood pressure on sepsis?
ax=sns.barplot(
    data = train_data,
    x = 'Sepsis',
    y = 'Blood_Pressure (mm Hg)',
    ci = None
)
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha = 'center',
        va = 'center'
    )
plt.title('Effect on blood pressure')
plt.xlabel('Sepsis')
plt.ylabel('Blood Pressure')
plt.show()

In [ ]:
# iv. What is the relationship between body mass index and sepsis?
ax=sns.barplot(
    data = train_data,
    x = 'Sepsis',
    y = 'Body_Mass_Index (weight in kg/(height in m)^2',
    ci = None
)
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha = 'center',
        va = 'center'
    )
plt.title('The relationship Between Body Mass on Sepsis')
plt.xlabel('Sepsis')
plt.ylabel('Body Mass')
plt.show()

In [ ]:
# v. Does one insurance health affect sepsis?
ax = sns.barplot(
    data = train_data,
    x = 'Sepsis',
    y = 'Insurance',
    ci = None
)
plt.title('Insurance vs Sepsis')
plt.xlabel('Sepsis')
plt.ylabel('Insurance')

for p in ax.patches:
    ax.annotate(
        f"{round(p.get_height(), 3)}",
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha = 'center',
        va = 'center'
    )
plt.show()

### Hypothesis Testing

In [ ]:
train_data = train_data.applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)

# Null Hypothesis (H₀): There is no significant relationship between the patient's medical data and the likelihood of developing sepsis.
# Alternative Hypothesis (H₁): There is a significant relationship between the patient's medical data and the likelihood of developing sepsis.

# Set the alpha value

alpha = 0.05

# Split the data into features and target
features = train_data[['Body_Mass_Index (weight in kg/(height in m)^2']]

# Get the target variable and convert it numerical
target = train_data[['Sepsis']]

# Intialize label encoder
le = LabelEncoder()
target = le.fit_transform(target)


# Get spearmanr correlation and p value of features vs target
corr, p_value = spearmanr(features, target)
print(f"The spearmanr p value of the data is: {p_value}")

# Decision making based on p-value
if p_value < alpha:
    print("Reject the null hypothesis (H₀): There is no significant relationship between the patient's medical data and the likelihood of developing sepsis.")
else:
    print("Fail to reject the null hypothesis (H₀): There is a significant relationship between the patient's medical data and the likelihood of developing sepsis.")

### Key Insights
1. There are no missing values in the dataset
2. There are no duplicate values in the data set
3. The Insurance has well distributed values with a minimum of zero (0) and a maximum of one (1)
4. All the features have a minimun value of zero (0)
5. The blood work result-3 has the greatest outlier with minimun value of zero (0) and a maximum value of 846
6. Reject the Null Hypothesis

### Data Preparation

In [ ]:
# Check is the data is balanced and visualize

class_sepsis = train_data['Sepsis'].value_counts().rename('Count_Sepsis').reset_index()                                               
print(class_sepsis) 

     Sepsis  Count_Sepsis
0  Negative           391
1  Positive           208


In [ ]:
# A Visual of the target Class Balance

ax = sns.countplot(
    data = train_data,
    x = 'Sepsis',
    palette = 'viridis'
)
plt.xlabel('Class Target')
plt.ylabel('Frequency')
plt.title('Count of Class Target')

# Annotate the figures for each category
for p in ax.patches:
    ax.annotate(
        f"{int(p.get_height())}",
        (p.get_x() + p.get_width()/2, p.get_height()),
        ha = 'center',
        va = 'center'
    )
plt.show()

In [ ]:
# From the analysis above, its clear the positive is minority class

minority_sepsis = train_data[train_data['Sepsis'] == 'Positive']
majority_sepsis = train_data[train_data['Sepsis'] == 'Negative']

# Resample the minority class to match the majority class size
minor_resampled = resample(minority_sepsis, replace=True, n_samples=len(majority_sepsis), random_state=42)

# Combine the resampled minority class with the majority class
train_model = pd.concat([majority_sepsis, minor_resampled])

# Check the class distribution in the resampled dataset
print(train_model['Sepsis'].value_counts())

Sepsis
Negative    391
Positive    391
Name: count, dtype: int64


### Split the Data Set into Features and Target Variables

In [ ]:
# Get the feature variables

X = train_model.drop(columns = ['Patient_ID', 'Sepsis'], axis = 1)

# Get the target variable and convert it into numeric

y = train_model['Sepsis'].apply(lambda x: 1 if x == 'Negative' else 0)

### Split the Data Set into Train and Validation Set

In [ ]:
# Split the data into train and validation set

X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 42)

In [ ]:
#check the size of the split data

(X_train.shape, y_train.shape), (X_val.shape, y_val.shape)

(((625, 9), (625,)), ((157, 9), (157,)))


In [ ]:
# Identify the Feature Columns that will need Transformation

feature_cols = X.columns
print(feature_cols)

Index(['Plasma_Glucose', 'Blood_Work_Result-1 (mu U/ml)',
       'Blood_Pressure (mm Hg)', 'Blood_Work_Result-2 (mm)',
       'Blood_Work_Result-3 (mu U/ml)',
       'Body_Mass_Index (weight in kg/(height in m)^2',
       'Blood_Work_Result-4 (mu U/ml)', 'Age', 'Insurance'],
      dtype='object')


### Create a Preprocessor

In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
        ('feature_columns', make_pipeline(
            SimpleImputer(strategy = 'median'),
            RobustScaler(),
            QuantileTransformer(output_distribution = 'normal')),
         feature_cols
        )
    ]
)

print(preprocessor)

ColumnTransformer(transformers=[('feature_columns',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('robustscaler',
                                                  RobustScaler()),
                                                 ('quantiletransformer',
                                                  QuantileTransformer(output_distribution='normal'))]),
                                 Index(['Plasma_Glucose', 'Blood_Work_Result-1 (mu U/ml)',
       'Blood_Pressure (mm Hg)', 'Blood_Work_Result-2 (mm)',
       'Blood_Work_Result-3 (mu U/ml)',
       'Body_Mass_Index (weight in kg/(height in m)^2',
       'Blood_Work_Result-4 (mu U/ml)', 'Age', 'Insurance'],
      dtype='object'))])


### Modeling

In [ ]:
# Define the Classification models

models = {
    "logistic_regression": LogisticRegression(),
    "decision_tree": DecisionTreeClassifier(),
    "random_forest": RandomForestClassifier(),
    "gradient_boosting": GradientBoostingClassifier(),
    "ada_boost": AdaBoostClassifier(),
    "extra_trees": ExtraTreesClassifier(),
    "K_nearest_neighbor": KNeighborsClassifier(),
    "bagging_classifier": BaggingClassifier()
}

In [ ]:
# Preprocess the data

X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

In [ ]:
# Initialize an empty dictionary to score evaluation scores
model_report = {}

# Loop through models, fit, and evaluate each model
for model_name, model_classifier in models.items():
    
    # Fit the model on the processed training data
    model_classifier.fit(X_train_processed, y_train)
    
    # Predict on the processed test data
    y_pred = model_classifier.predict(X_val_processed)
    
    # Get the evaluation score for each metric
    F1_Score = f1_score(y_val, y_pred)
    Precision_Score = precision_score(y_val, y_pred)
    Recall_Score = recall_score(y_val, y_pred)
    Accuracy_Score = accuracy_score(y_val, y_pred)
    Confusion_Matrix = confusion_matrix(y_val, y_pred)
    Predictions = y_pred
    
    # Store the evaluation score for each model
    model_report[model_name] = {
        'f1_score': F1_Score,
        'precision_score': Precision_Score,
        'recall_score': Recall_Score,
        'accuracy_score': Accuracy_Score,
        'confusion_matrix': Confusion_Matrix,
        'predictions': Predictions
    }

# Convert the model_reports dictionary into a dataframe
eval_report = pd.DataFrame(model_report).transpose()

# Sort the dataframe in descending order by the f1_score column
eval_report_df = eval_report.sort_values('f1_score', ascending = False)

# Display the dataframe
print(eval_report_df)

                     f1_score precision_score recall_score accuracy_score  \
extra_trees          0.910256        0.910256     0.910256       0.910828   
random_forest        0.894737        0.918919     0.871795       0.898089   
decision_tree        0.885906        0.929577     0.846154        0.89172   
gradient_boosting    0.874172         0.90411     0.846154       0.878981   
bagging_classifier   0.864865        0.914286     0.820513       0.872611   
ada_boost            0.824324        0.871429     0.782051       0.834395   
logistic_regression  0.753425        0.808824     0.705128       0.770701   
K_nearest_neighbor   0.737589        0.825397     0.666667       0.764331   

                         confusion_matrix  \
extra_trees            [[72, 7], [7, 71]]   
random_forest         [[73, 6], [10, 68]]   
decision_tree         [[74, 5], [12, 66]]   
gradient_boosting     [[72, 7], [12, 66]]   
bagging_classifier    [[73, 6], [14, 64]]   
ada_boost             [[70, 9], [17,

In [ ]:
# Create a visualization of confusion matrix

def confusion_matrix_plot(confusion_matrix, model):
    plt.figure(figsize = (5, 4))
    sns.heatmap(
        confusion_matrix,
        annot = True,
        cmap = "coolwarm",
        fmt = 'd',
        xticklabels = ["Positive", "Negative"],
        yticklabels = ["Positive", "Negative"]
    )
    plt.title(f"Confusion_Matrix for: {model}")
    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")
    
for model in model_report.keys():
    confusion_matrix = model_report[model]["confusion_matrix"]
    confusion_matrix_plot(confusion_matrix, model)

In [ ]:
# Display the classification report

for model_name, model_classifier in models.items():
    
    print(f"Classification Report for {model_name}")
    print(classification_report(y_val, y_pred, target_names = ["Positive", "Negative"]))
    print("=="*30 + "\n")

Classification Report for logistic_regression
              precision    recall  f1-score   support

    Positive       0.84      0.92      0.88        79
    Negative       0.91      0.82      0.86        78

    accuracy                           0.87       157
   macro avg       0.88      0.87      0.87       157
weighted avg       0.88      0.87      0.87       157


Classification Report for decision_tree
              precision    recall  f1-score   support

    Positive       0.84      0.92      0.88        79
    Negative       0.91      0.82      0.86        78

    accuracy                           0.87       157
   macro avg       0.88      0.87      0.87       157
weighted avg       0.88      0.87      0.87       157


Classification Report for random_forest
              precision    recall  f1-score   support

    Positive       0.84      0.92      0.88        79
    Negative       0.91      0.82      0.86        78

    accuracy                           0.87       157


### Hyperparameter Tuning using RandomizedSearchCV

In [ ]:
# Hyperparameter grid for each model

param_grids = {
    "logistic_regression": {
        'C': [0.1, 1, 10, 100],  # Regularization strength
        'solver': ['newton-cg', 'lbfgs', 'liblinear'],  # Optimization algorithms
        'penalty': ['l2'],  # L2 regularization
        'max_iter': [100, 200, 500]
    },
    "decision_tree": {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': [5, 10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 5],
        'max_features': ['auto', 'sqrt', 'log2', None]
    },
    "random_forest": {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False],
        'max_features': ['auto', 'sqrt', 'log2']
    },
    "gradient_boosting": {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 0.8, 1.0],
        'max_depth': [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    "ada_boost": {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1.0],
        'algorithm': ['SAMME', 'SAMME.R']
    },
    "extra_trees": {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    },
    "K_nearest_neighbor": {
        'n_neighbors': [3, 5, 11, 19],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    },
    "bagging_classifier": {
        'n_estimators': [10, 50, 100],
        'max_samples': [0.5, 0.7, 1.0],
        'max_features': [0.5, 0.7, 1.0],
        'bootstrap': [True, False]
    }
}

In [ ]:
# Create empty dataframe to store the results
rand_table = pd.DataFrame(columns = ["model", "best_params", "best_scores"])

# Initialize an empty dictionary to hold the RandomizedSearchCV results for each model
rand_searches_tuned = {}

# Initialize a dictionary to hold the best model
best_model_rand = {}

# Create a stratified object
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

# Iterate over models and apply RandomizedSearchCV for hyperparameter tuning

for model_name, model_classifier in models.items():
    
    # Get the parameter grid for the model
    param_grid = param_grids.get(model_name, {})
    
    # Set up the RandomizedSearchCV with 5-fold cross-validation
    rand_search_tuned = RandomizedSearchCV(
        model_classifier,
        param_distributions=param_grid,
        n_iter=20,
        cv=skf,
        scoring='f1',
        verbose = 0,
        n_jobs =- 1,
        random_state = 42
    )
    
    # Fit RandomizedSearchCV
    rand_search_tuned.fit(X_train_processed, y_train)
    
    # Store the randomized search object in the dictionary
    rand_searches_tuned[model] = rand_search_tuned
    
    # Add the results to the dataframe
    best_params = rand_search_tuned.best_params_
    best_score = rand_search_tuned.best_score_
    
    rand_table.loc[len(rand_table)] = [model_name, best_params, best_score]
    
    # Store the best model in best_model_rand dictionary
    best_model_rand[model_name] = rand_search_tuned.best_estimator_
    
    
# Sort the rand table with the best score
rand_table = rand_table.sort_values(by = "best_scores", ascending = False).reset_index(drop = True)

# Display the rand table
print(rand_table)

                 model  \
0        random_forest   
1    gradient_boosting   
2          extra_trees   
3   bagging_classifier   
4        decision_tree   
5            ada_boost   
6   K_nearest_neighbor   
7  logistic_regression   

                                                                                                                           best_params  \
0    {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}   
1        {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 10, 'learning_rate': 0.2}   
2                            {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': False}   
3                                                    {'n_estimators': 100, 'max_samples': 0.7, 'max_features': 0.5, 'bootstrap': True}   
4  {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max

### Model Evaluation

In [ ]:
# Create a copy of test data set
test_df = test_data.copy()

### Exploratory Data Analysis (EDA) of Test Data Set

In [ ]:
# Check the info of the test data

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         169 non-null    object 
 1   PRG        169 non-null    int64  
 2   PL         169 non-null    int64  
 3   PR         169 non-null    int64  
 4   SK         169 non-null    int64  
 5   TS         169 non-null    int64  
 6   M11        169 non-null    float64
 7   BD2        169 non-null    float64
 8   Age        169 non-null    int64  
 9   Insurance  169 non-null    int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 13.3+ KB


In [ ]:
# Check for missing values of the test set

test_df.isna().sum()

ID           0
PRG          0
PL           0
PR           0
SK           0
TS           0
M11          0
BD2          0
Age          0
Insurance    0
dtype: int64


In [ ]:
# Rename the columns to give descriptive meaning

cols = {
    'ID': 'Patient_ID',
    'PRG': 'Plasma_Glucose',
    'PL': 'Blood_Work_Result-1 (mu U/ml)',
    'PR': 'Blood_Pressure (mm Hg)',
    'SK': 'Blood_Work_Result-2 (mm)',
    'TS': 'Blood_Work_Result-3 (mu U/ml)',
    'M11': 'Body_Mass_Index (weight in kg/(height in m)^2',
    'BD2': 'Blood_Work_Result-4 (mu U/ml)',
    'Sepssis' : 'Sepsis'
}

test_df.rename(columns = cols, inplace = True)

# Check the data
print(test_df.head())

  Patient_ID  Plasma_Glucose  Blood_Work_Result-1 (mu U/ml)  \
0  ICU200609               1                            109   
1  ICU200610               1                            108   
2  ICU200611               6                             96   
3  ICU200612               1                            124   
4  ICU200613               7                            150   

   Blood_Pressure (mm Hg)  Blood_Work_Result-2 (mm)  \
0                      38                        18   
1                      88                        19   
2                       0                         0   
3                      74                        36   
4                      78                        29   

   Blood_Work_Result-3 (mu U/ml)  \
0                            120   
1                              0   
2                              0   
3                              0   
4                            126   

   Body_Mass_Index (weight in kg/(height in m)^2  \
0                    

In [ ]:
# Thorough check the test set

for col in test_df.columns:
    print(f"{col} column is of {test_df[col].dtype} data type\n")
    
    # Check if applying np.isnan will work
    try:
        np.isnan(test_df[col])
    except TypeError:
        print(f"np.isnan() is not applicable to {col} column")

Patient_ID column is of object data type

np.isnan() is not applicable to Patient_ID column
Plasma_Glucose column is of int64 data type

Blood_Work_Result-1 (mu U/ml) column is of int64 data type

Blood_Pressure (mm Hg) column is of int64 data type

Blood_Work_Result-2 (mm) column is of int64 data type

Blood_Work_Result-3 (mu U/ml) column is of int64 data type

Body_Mass_Index (weight in kg/(height in m)^2 column is of float64 data type

Blood_Work_Result-4 (mu U/ml) column is of float64 data type

Age column is of int64 data type

Insurance column is of int64 data type



In [ ]:
# Drop the unnecessary columns from the test_set dataframe

test_set = test_df.drop(columns = ["Patient_ID"], axis = 1)

### Use Model on the Test Dataframe

In [ ]:
# Create an empty dictionary to store the best estimators
best_estimators = {}

# Create an empty dictionary to store the y tests results
y_test = {}

# Iterate over the top four best models
for model in ['gradient_boosting', 'random_forest', 'extra_trees', 'bagging_classifier']:
    if model in best_model_rand:
        best_estimators[model] = best_model_rand[model]
    else:
        print(f"{model} does not exist")

In [ ]:
for model, model in best_estimators.items():
    y_test[model] = model.predict(test_set)

In [ ]:
# Print the predictions for each model
for model_name, predictions in y_test.items():
    print(predictions)

[1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0
 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 0 1
 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 1
 1 1 0 1 0 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1 0
 1 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0

In [ ]:
# Import the confusion matrix for use so that it can be updated
from sklearn.metrics import confusion_matrix

# Initialize Results dictionary
result_test = {}

# Loop through rand_searches_tuned items
for model, rand_search_tuned in best_model_rand.items():
    
    # Ensure only top four best performing models are processed
    if model not in ["gradient_boosting", "random_forest",  "extra_trees", "bagging_classifier"]:
        continue
    
    # Predict on evaluation data
    y_test_pred = rand_search_tuned.predict(test_set)
    
    # Calculate other metrics
    accuracy = accuracy_score(predictions, y_test_pred)
    f1 = f1_score(predictions, y_test_pred)
    precision = precision_score(predictions, y_test_pred)
    recall = recall_score(predictions, y_test_pred)
    conf_matrix = confusion_matrix(predictions, y_test_pred)
    # y_test_prob = rand_search_tuned(predictions, y_test_pred)
    roc_auc = roc_auc_score(predictions, y_test_pred)
    
    # Store the results in eval_results_tuned dictionary
    result_test[model] = {
        'accuracy_score' : accuracy,
        'f1_score' : f1,
        'precision_score' : precision,
        'recall_score' : recall,
        'confusion_matrix' : conf_matrix,
        # 'y_test_prob' : y_test_prob,
        'roc_auc_score' : roc_auc
    }
    
# Convert results into a dataframe
scores_test = pd.DataFrame(result_test).transpose()

# Sort the dataframe by roc_auc column
scores_test_df = scores_test.sort_values(by = "f1_score", ascending = False)

# Display the sorted evaluation scores
print("Model Scores Evaluations on Test Data:")
print(scores_test_df)

Model Scores Evaluations on Test Data:
                   accuracy_score f1_score precision_score recall_score  \
bagging_classifier            1.0      1.0             1.0          1.0   
random_forest            0.544379      0.0             0.0          0.0   
gradient_boosting        0.047337      0.0             0.0          0.0   
extra_trees              0.544379      0.0             0.0          0.0   

                      confusion_matrix roc_auc_score  
bagging_classifier  [[92, 0], [0, 77]]           1.0  
random_forest       [[92, 0], [77, 0]]           0.5  
gradient_boosting   [[8, 84], [77, 0]]      0.043478  
extra_trees         [[92, 0], [77, 0]]           0.5  


In [ ]:
# Create a visualization of the test confusion matrix

for model in result_test.keys():
    conf_mat = result_test[model]["confusion_matrix"]
    confusion_matrix_plot(conf_mat, model)

**Conclusion:**

The created model helps the medical facilities to predict Sepsis disease, which will enhance actions to have early detection of patients who can potentially contract the diease and, consequently, reduce the overall ineffcetion rates. Since Sepsis is caused by a range of factors, one has to focus on them, for instance, patients can be having regular medical check-ups for their blood pressure and body mass index among others. 

### Saving the Cleaned Data, Model and Preprocessor

In [ ]:
# saving my clean data frame 'traindata'

train_data.to_csv('train_sepsis_data.csv', index=False)
test_set.to_csv('test_sepsis_data.csv', index = False)

In [ ]:
# Display the dataframe that have scored the hyperparametered models
print(rand_table)

                 model  \
0        random_forest   
1    gradient_boosting   
2          extra_trees   
3   bagging_classifier   
4        decision_tree   
5            ada_boost   
6   K_nearest_neighbor   
7  logistic_regression   

                                                                                                                           best_params  \
0    {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}   
1        {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 10, 'learning_rate': 0.2}   
2                            {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': False}   
3                                                    {'n_estimators': 100, 'max_samples': 0.7, 'max_features': 0.5, 'bootstrap': True}   
4  {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max

# Save each model pipeline as a pickle file

In [ ]:
# Create and save the pipelines for the best models
model_pipelines = {}

for model_name, best_model in best_model_rand.items():
    # Combine preprocessor and model into a pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),  # Add the preprocessor
        ('classifier', best_model)  # Add the best model
    ])
    
    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)
    
    # Save the pipeline as a pickle file
    pipeline_filename = f"{model_name}_pipeline.pkl"
    with open(pipeline_filename, 'wb') as file:
        pickle.dump(pipeline, file)
    
    # Store the pipeline for further use
    model_pipelines[model_name] = pipeline
    print(f"Saved {model_name} pipeline to {pipeline_filename}")

# Save individual models (optional, since pipelines include models)
for model_name, model in best_model_rand.items():
    model_filename = f"{model_name}_model.pkl"
    joblib.dump(model, model_filename)
    print(f"Saved {model_name} model to {model_filename}")

# Loading the pipelines or models for inference
# Example: Loading a saved pipeline
pipeline_path = "gradient_boosting_pipeline.pkl"
with open(pipeline_path, 'rb') as file:
    loaded_pipeline = pickle.load(file)

# Predict using the loaded pipeline
test_predictions = loaded_pipeline.predict(test_set)
print(test_predictions)

Saved logistic_regression pipeline to logistic_regression_pipeline.pkl
Saved decision_tree pipeline to decision_tree_pipeline.pkl
Saved random_forest pipeline to random_forest_pipeline.pkl
Saved gradient_boosting pipeline to gradient_boosting_pipeline.pkl
Saved ada_boost pipeline to ada_boost_pipeline.pkl
Saved extra_trees pipeline to extra_trees_pipeline.pkl
Saved K_nearest_neighbor pipeline to K_nearest_neighbor_pipeline.pkl
Saved bagging_classifier pipeline to bagging_classifier_pipeline.pkl
Saved logistic_regression model to logistic_regression_model.pkl
Saved decision_tree model to decision_tree_model.pkl
Saved random_forest model to random_forest_model.pkl
Saved gradient_boosting model to gradient_boosting_model.pkl
Saved ada_boost model to ada_boost_model.pkl
Saved extra_trees model to extra_trees_model.pkl
Saved K_nearest_neighbor model to K_nearest_neighbor_model.pkl
Saved bagging_classifier model to bagging_classifier_model.pkl
[1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 0